In [7]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [8]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'home appliances' # topic is factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(10) # first 10 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/ferdinandk/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 10 documents.


In [9]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


Land, a Cleveland-based electric mobility and portable battery manufacturer, recently secured $3 million in funding. Founded in 2020 by Scott Colosimo, Land focuses on the importance of energy continuity and mobility in modern life. The company offers a unique e-Moto ecosystem with a portable power platform called CORE. This innovative technology allows users to distribute energy from the battery, serving as a portable power source both on and off the grid. By using the 'Power Tap' accessory, consumers can charge various devices like phones, laptops, and household appliances through USB-C ports and standard plugs.

The undisclosed backers of Land's recent funding round have enabled the company to further its mission. In September 2023, Land closed a Series A funding round, raising $7 million. This financial boost has facilitated the upgrade of Land's 30,000-square-foot manufacturing facility and the expansion of its team. As a result, the company has been able to produce more new vehic

Software Defined Automation, a company in the field of factory automation, recently secured $10 million in seed funding. The funding round was led by Insight Partners, with participation from Baukunst VC, Fly Ventures, and First Momentum. Insight Partners' Managing Director Jon Rosenbaum and Baukunst's Co-Founder and General Partner Axel Bichara will now join the company's Board of Directors as part of the transaction.

["Land, a Cleveland-based electric mobility and portable battery manufacturer, recently secured $3 million in funding. Founded in 2020 by Scott Colosimo, Land focuses on the importance of energy continuity and mobility in modern life. The company offers a unique e-Moto ecosystem with a portable power platform called CORE. This innovative technology allows users to distribute energy from the battery, serving as a portable power source both on and off the grid. By using the 'Power Tap' accessory, consumers can charge various devices like phones, laptops, and household app

In [10]:
Problem = """Title: Sustainable Water Appliance Integration

Explanation: As consumer awareness and regulatory requirements around sustainability increase, there's a growing demand for home appliances that not only reduce water and energy consumption but also integrate seamlessly with home water treatment systems like softeners and purifiers. The problem lies in the lack of interconnected appliances that can communicate with water treatment systems to optimize their operation based on water quality. For instance, washing machines and dishwashers that adjust their cycles and detergent usage based on real-time feedback from a connected water softener system could significantly enhance efficiency and sustainability. This innovation requires a blend of IoT technology, advances in material science for developing more efficient filters, and AI for predictive analytics and optimization.

User group: Environmentally-conscious homeowners

Pain point: This group struggles with the inefficiency and increased environmental footprint of using appliances that operate at a standard setting, regardless of the varying quality of input water, leading to unnecessary waste and higher operational costs."""

print(Problem)

Title: Sustainable Water Appliance Integration

Explanation: As consumer awareness and regulatory requirements around sustainability increase, there's a growing demand for home appliances that not only reduce water and energy consumption but also integrate seamlessly with home water treatment systems like softeners and purifiers. The problem lies in the lack of interconnected appliances that can communicate with water treatment systems to optimize their operation based on water quality. For instance, washing machines and dishwashers that adjust their cycles and detergent usage based on real-time feedback from a connected water softener system could significantly enhance efficiency and sustainability. This innovation requires a blend of IoT technology, advances in material science for developing more efficient filters, and AI for predictive analytics and optimization.

User group: Environmentally-conscious homeowners

Pain point: This group struggles with the inefficiency and increased 

# Answering questions on the results with OpenAI GPT

In [11]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f"""You are part of a brainstorming session that wants to come up with a solution to a specific problem. You customer handed you the following problem:
       
        {Problem}
        
        Now you try to figure out a solution in a brainstorming session with the Walt Disney Method: 
        It is a tool for brainstorming ideas. It involves using 3 sequential roles, namely the Dreamer, the Realist, and the Critic.
        The Dreamer: you focus on free association and brainstorming of ideas. Anything goes here. It's your chance to let your imagination run wild! Be enthusiastic and approach the task with questions such as “What would be nice?”, “What would be the optimal situation?” or “What would I wish for?” Assume that everything is possible.
        The Realist: Be realistic and decide which of your ideas are actually possible or practical. What would you need to do in the real world in order to make them happen? Take a pragmatic approach and ask which materials, which knowledge, which techniques there are with which the thoughts of the dreamer can be realised.
        The Critic: Examine proposals and its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. It is about identifying opportunities and risks, pointing out limits and gaps, and expressing constructive and always positive criticism. Try to identify possible sources of error.
        
        Your work is highly crucial and you are responsible of coming up with the next best innovation.
        """

#Dreamer prompt (idea: come up with crazy ideas)
dreamer_solutions = {}
i = 0

user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {summary_list}
    ```    
  
    Imagine you are the Dreamer. 
    
    Take a deep breath and work on this problem step by step. 

    
    Please develop 5 really specific ideas/visions on how this problem can be solved. The solutions should consider the following points:
    - You can let your imagination run wild and you do not care about possible limitations. It's more important the idea is new then it can really work.  
    - They should not be too general. 
    - The solution should not already exist on the market and be novel. 
    - The ideas should not be too similar to each other
    
    For the solution, please write only 40 words. It should contain the following contents:
    1) Your solution in max 3 words (title)
    2) Your solution summarized in 1 sentence
    3) Your solution from the customer perspective (customer journey: how does it work for the customer) 
    4) Why the solution is kind of new (not existing on the market)
    5) Please provide some sources of inspiration that you used from the document excerpts.


    
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
dreamer_solutions[i] = completion.choices[0].message
i += 1


#Realist prompt (idea: evaluate if it is possible and what would be needed)
realist_solutions = {}

user_prompt1 = f"""

    Imagine you are the Realist. 
    
    Take a deep breath and work on this problem step by step. 

    
    Consider the following solutions: {dreamer_solutions}.
    
    Please give me the 3 clearly speperated pieces of information for each of the solutions (in total 50 words max):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 1 detail": What would you need to do in the real world in order to make the solution happen? Answer which knowledge, which techniques there are with which the solutions can be realised. Explain in 5 sentences. 
    4) "Evaluation criteria 1": Is the solution actually possible? Answer can be 1,2 or 3 (1 = not possible, 2 = maybe possible, 3 = possible) (be the most critical person ever. Only give max. 1 of the 3 solution a 3)

    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt1}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
realist_solutions[0] = completion.choices[0].message


#Critics prompt (idea: evaluate if there are a lot potential sources of error)
critics_solutions = {}

user_prompt2 = f"""

    Imagine you are the Critic. 
    
    Consider the following solutions: {realist_solutions}.
    
    Take a deep breath and work on this problem step by step. 
    
    Please give me the following 4 clearly seperated pieces of information for each solution (40 words in total):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 2 details": Identify possible sources of error. Look for risks, pointing out limits and gaps, and expressing constructive criticism. Examine its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. (1 sentence only)
    4) "Evaluation criteria 2": Does it have many potential sources of errors? Answer can be 1,2 or 3 (1 = many sources of error, 2 = some sources of error, 3 = few sources of error) (be the most critical person ever. Only give max. 1 of the 3 solution a 3) (1 sentence)
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt2}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
critics_solutions[0] = completion.choices[0].message


#Summary

summary_solutions = {}

user_prompt3 = f"""

    Imagine you should give a summary to a high executive manager about a brainstorming session. 
    
    During the brainstorming session your team came up with the following ideas: {dreamer_solutions}. 
    For each idea you considered how feasible it is {realist_solutions}. 
    Last, you evaluated the amount of potential sources of errors: {critics_solutions} 
    
    Take a deep breath and work on this problem. 
    
    Please summarize in one paragraph for each solution, with the following content:
    
    “Name”: Solution name (5 words max)
    “Solved Problem”: 1 sentence summary of the problem that is being solved 
    “Solution”: Provide a short summary of the solution and how it works (4-5 sentences)
    “Impact”: How does the solution solve the specific problem (2-3 sentences)
    “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution  (4-5 sentences)
    “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions[0] = completion.choices[0].message



1) **Smart Water Guardian**

- A device that dynamically adjusts water quality for each appliance.
- Homeowners connect Smart Water Guardian to their main water supply and appliances, which then customizes water treatment for each use, optimizing efficiency.
- Introduces real-time, appliance-specific water treatment adjustments.
- Inspired by e2ip technologies' focus on smart interfaces and Aliste Technologies' home automation solutions.

2) **AquaSync Circuits**

- Appliance-embedded water treatment circuits.
- Embedding AquaSync Circuits in appliances allows them to self-regulate water usage and treatment, based on load and water quality, directly.
- Appliances independently adapt to water conditions without external devices.
- Draws inspiration from e2ip technologies' embedded systems and SPAN's energy management systems.

3) **EcoDetergent Creator**

- On-demand, eco-friendly detergent production.
- Users fill the EcoDetergent Creator with base ingredients; it synthesizes customize

In [12]:
# Initial rating of the solutions

initial_rating = {}

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

Please take all solutions from the following text: {summary_solutions}
First, repeat the following points for the first solution:
    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”

Then, rate the first solution according to the following criteria. Please justify your ratings in max one sentence. Please be very critical in your ratings and only give a 3 out of 3 rating when it is very good:   

    - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. (1 = low performance, 2 = medium performance, 3 = high performance)
    - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution? (1 = low scalability, 2 = medium scalability, 3 = high scalability)
    - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment. (1 = low cost-effectiveness, 2 = medium cost-effectiveness 3 = high cost-effectiveness)
    - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already? (1 = low innovativeness, 2 = medium innovativeness 3 = high innovativeness)
    - Uniqueness: The solutions contained in the list should differ substantially and not be similar. (1 = low uniqueness, 2 = medium uniqueness 3 = high uniqueness)
    - Overall Rating: Sum of all the previous ratings, out of 15

Now, repeat these steps for all solutions.

"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating[0] = completion.choices[0].message



### Smart Water Guardian

- **Name**: Smart Water Guardian
- **Solved Problem**: Inefficient water usage and treatment in home appliances.
- **Solution**: Dynamically adjusts water quality for each appliance by connecting to the main water supply and appliances, thus customizing treatment based on real-time needs.
- **Impact**: Appliances operate more efficiently, conserving water and reducing environmental impact, leading to significant savings on operational costs and enhancing the lifespan of appliances.
- **Technology**: Utilizes IoT technology with sensors to analyze water quality in real-time and communicate with home appliances to adjust water treatment, leveraging advanced machine learning algorithms.
- **Sources**: Inspired by e2ip technologies' smart interfaces and Aliste Technologies' home automation solutions.

**Solution**: UAL proposes a universal programming language designed for all factory automation systems, enabling engineers to write code once, which is then transla

In [10]:
user_prompt5 = f""""

Please take all the solutions and the corresponding ratings from: {initial_rating}

Take the three solutions with the highest overall rating out of 15, sorting them in descending rating order.

Outline the following points for these solution from the following text: {summary_solutions}:
    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”


"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt5}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability

### 1. Universal Automation Language (UAL)

- **Name**: Universal Automation Language (UAL)
- **Solved Problem**: This solution addresses the challenge of integrating diverse factory automation systems due to incompatible programming languages.
- **Solution**: UAL proposes a universal programming language designed for all factory automation systems, enabling engineers to write code once, which is then translated into the specific languages of various equipment and systems. It aims to create a seamless bridge between the myriad of languages and protocols currently in use, facilitating easier and more efficient interoperability among different automation technologies. The end goal is to significantly reduce the time and effort required for system integration and to enhance operational efficiency.
- **Impact**: By standardizing the programming language across different systems, UAL minimizes the need for manual intervention and reduces the likelihood of errors during the integration proce